## Finding the global minimum of the Rosenbrock function
---

> In mathematical optimization, the Rosenbrock function is a non-convex function, introduced by Howard H. Rosenbrock in 1960, which is used as a performance test problem for optimization algorithms. It is also known as Rosenbrock's valley or Rosenbrock's banana function.
>
> The global minimum is inside a long, narrow, parabolic shaped flat valley. To find the valley is trivial. To converge to the global minimum, however, is difficult.

<p style='text-align: right;'> -- <a href=https://en.wikipedia.org/wiki/Rosenbrock_function>Wikipedia</a> </p>

### The OpenMDAO way
---
#### Create a `Component` to represent the Rosenbrock function

In [1]:
import numpy as np
import openmdao.api as om

class RosenbrockComp(om.ExplicitComponent):
    """
    nth dimensional Rosenbrock function, array input and scalar output
    global minimum at f(1,1,1...) = 0
    """
    def initialize(self):
        self.options.declare('order', types=int, default=2, desc='dimension of input.')

    def setup(self):
        n = self.options['order']
        
        self.add_input('x', np.zeros(n))
        self.add_output('y', 0.0)

    def compute(self, inputs, outputs):
        x = inputs['x']
        
        n = len(x)
        s = 0
        for i in range(n - 1):
            s += 100 * (x[i + 1] - x[i] * x[i]) ** 2 + (1 - x[i]) ** 2
        
        outputs['y'] = s

#### Create a model by putting the `Component` in a `Group` and specifying the design variables and objective

In [2]:
ORDER = 6  # dimension of problem

span = 2   # upper and lower limits
lower_bound = -span*np.ones(ORDER)
upper_bound = span*np.ones(ORDER)

my_model = om.Group()
my_model.add_subsystem('rosenbrock', RosenbrockComp(order=ORDER))
my_model.add_design_var('rosenbrock.x', lower=lower_bound, upper=upper_bound)
my_model.add_objective('rosenbrock.y')

#### Create a `Problem` to run the model using your choice of optimizers as the `Driver`

In [3]:
p = om.Problem(model=my_model, driver=om.DifferentialEvolutionDriver(max_gen=500))
p.setup()

fail = p.run_driver()
assert fail is False

#### The global minimum is found at [1, 1, 1, ...]

In [4]:
print('xopt:', p['rosenbrock.x'])
print('fopt:', p['rosenbrock.y'])
print('model execution count:', p.driver.iter_count)

xopt: [0.99991615 0.99997196 0.99986423 0.99989539 0.99995877 0.99983555]
fopt: [8.90398716e-06]
model execution count: 60121





### The `pycma` way
---
#### Using the functional interface

In [5]:
import cma

def rosenbrock(x):
    n = len(x)
    s = 0
    for i in range(n - 1):
        s += 100 * (x[i + 1] - x[i] * x[i]) ** 2 + (1 - x[i]) ** 2
    return s

# NOTE: the Rosenbrock function is also available in the "fitness functions" library
# rosenbrock = cma.ff.rosen

# find global minimum via the fmin function
res = cma.fmin(rosenbrock, [-1]*ORDER, 0.01,
               options={'ftarget':1e-6, 'bounds':[lower_bound, upper_bound], 'seed': 3})

(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=3, Tue Oct 27 21:24:32 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 2.001843015092397e+03 1.0e+00 8.78e-03  8e-03  9e-03 0:00.0
    2     18 1.986125694620930e+03 1.4e+00 8.48e-03  8e-03  9e-03 0:00.0
    3     27 1.988866857733969e+03 1.5e+00 7.68e-03  7e-03  8e-03 0:00.0
  100    900 2.372114162584607e+00 8.1e+00 8.61e-02  1e-02  5e-02 0:00.6
  200   1800 1.088872415974996e-01 2.6e+01 1.29e-01  6e-03  4e-02 0:01.5
  266   2394 6.637172252308916e-07 5.3e+01 4.33e-03  5e-05  9e-04 0:01.7
termination on ftarget=1e-06 (Tue Oct 27 21:24:35 2020)
final/bestever f-value = 6.720343e-07 6.637172e-07
incumbent solution: [0.999982151058333, 0.9999866455805502, 0.9999825497485988, 0.9999100175543002, 0.9998658254305461, 0.9997588042973703]
std deviation: [5.3875484003553336e-05, 7.218132721595697e-05, 0.00011048916508625072, 0.0002242804956513507, 0.0004619327772433858, 0.0009076162191461554]


#### The global minimum is found at [1, 1, 1, ...]

In [6]:
print('xopt:', res[0])
print('fopt:', res[1])
print('function execution count:', res[2])

xopt: [0.99997403 0.99994719 0.9999355  0.9998314  0.99970865 0.99942561]
fopt: 6.637172252308916e-07
function execution count: 2390


### Use `pycma` as top level interface over OpenMDAO
---
#### Wrap an OpenMDAO `Component` for use with the `pycma` functional interface

In [7]:
comp = RosenbrockComp(order=ORDER)

def rosenbrock(x):
    inputs = {'x': x}
    outputs = {}
    comp.compute(inputs, outputs)
    return outputs['y']
    
# find global minimum via the fmin function
res = cma.fmin(rosenbrock, [-1]*ORDER, 0.01,
               options={'ftarget':1e-6, 'bounds':[lower_bound, upper_bound], 'seed': 3})

(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=3, Tue Oct 27 21:24:35 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 2.001843015092397e+03 1.0e+00 8.78e-03  8e-03  9e-03 0:00.0
    2     18 1.986125694620930e+03 1.4e+00 8.48e-03  8e-03  9e-03 0:00.0
    3     27 1.988866857733969e+03 1.5e+00 7.68e-03  7e-03  8e-03 0:00.0
  100    900 2.372114162584607e+00 8.1e+00 8.61e-02  1e-02  5e-02 0:00.4
  200   1800 1.088872415974996e-01 2.6e+01 1.29e-01  6e-03  4e-02 0:00.9
  266   2394 6.637172252308916e-07 5.3e+01 4.33e-03  5e-05  9e-04 0:01.2
termination on ftarget=1e-06 (Tue Oct 27 21:24:37 2020)
final/bestever f-value = 6.720343e-07 6.637172e-07
incumbent solution: [0.999982151058333, 0.9999866455805502, 0.9999825497485988, 0.9999100175543002, 0.9998658254305461, 0.9997588042973703]
std deviation: [5.3875484003553336e-05, 7.218132721595697e-05, 0.00011048916508625072, 0.0002242804956513507, 0.0004619327772433858, 0.0009076162191461554]


#### The global minimum is found at [1, 1, 1, ...]

In [8]:
print('xopt:', res[0])
print('fopt:', res[1])
print('component execution count:',res[2])

xopt: [0.99997403 0.99994719 0.9999355  0.9998314  0.99970865 0.99942561]
fopt: 6.637172252308916e-07
component execution count: 2390


#### Wrap an OpenMDAO `Problem` for use with the `pycma` functional interface

In [9]:
def rosenbrock(x):
    p['rosenbrock.x'] = x
    p.run_model()
    return p['rosenbrock.y'][0]
    
# find global minimum via the fmin function
res = cma.fmin(rosenbrock, [-1]*ORDER, 0.01,
               options={'ftarget':1e-6, 'bounds':[lower_bound, upper_bound], 'seed': 3})

(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=3, Tue Oct 27 21:24:37 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 2.001843015092397e+03 1.0e+00 8.78e-03  8e-03  9e-03 0:00.0
    2     18 1.986125694620930e+03 1.4e+00 8.48e-03  8e-03  9e-03 0:00.0
    3     27 1.988866857733969e+03 1.5e+00 7.68e-03  7e-03  8e-03 0:00.0
  100    900 2.372114162584607e+00 8.1e+00 8.61e-02  1e-02  5e-02 0:00.9
  200   1800 1.088872415974996e-01 2.6e+01 1.29e-01  6e-03  4e-02 0:01.8
  266   2394 6.637172252308916e-07 5.3e+01 4.33e-03  5e-05  9e-04 0:03.0
termination on ftarget=1e-06 (Tue Oct 27 21:24:41 2020)
final/bestever f-value = 6.720343e-07 6.637172e-07
incumbent solution: [0.999982151058333, 0.9999866455805502, 0.9999825497485988, 0.9999100175543002, 0.9998658254305461, 0.9997588042973703]
std deviation: [5.3875484003553336e-05, 7.218132721595697e-05, 0.00011048916508625072, 0.0002242804956513507, 0.0004619327772433858, 0.0009076162191461554]


#### The global minimum is found at [1, 1, 1, ...]

In [10]:
print('xopt:', res[0])
print('fopt:', res[1])
print('model execution count:', res[2])

xopt: [0.99997403 0.99994719 0.9999355  0.9998314  0.99970865 0.99942561]
fopt: 6.637172252308916e-07
model execution count: 2390
